In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import cv2

In [2]:
# edges
def edges(n, orient):
    edges = np.ones((2*n, 2*n, 3))
    
    if orient == 'vert':
        for i in range(0, 2*n):
            edges[i][n: 2*n] *= -1
    elif orient == 'horiz':
        edges[n: 2*n] *= -1
    
    return edges

In [3]:
# Apply one filter defined by parameters W and single slice
def conv_single_step(a_slice_prev, W):
    s = W * a_slice_prev
    Z = np.sum(s)
    Z = np.abs(Z)
    
    return Z

In [4]:
# Full edge filter
def conv_forward(A_prev, W, hparameters):
    m = len(A_prev)
    (f, f, n_C) = W.shape
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    Z = list()
    flag = 0
    z_max = hparameters['z_max']
    
    if len(z_max) == 0:
        z_max = list()
        flag = 1
    
    for i in range(m):
        
        (x0, x1, x2) = A_prev[i].shape
        A_prev_pad = A_prev[i][ 
                            int(x0 / 4) : int(x0 * 3 / 4), 
                            int(x1 / 4) : int(x1 * 3 / 4), 
                            :]
        
        (n_H_prev, n_W_prev, n_C_prev) = A_prev_pad.shape
        n_H = int((n_H_prev - f + 2*pad) / stride) + 1
        n_W = int((n_W_prev - f + 2*pad) / stride) + 1
        z = np.zeros((n_H, n_W))
        
        a_prev_pad = A_prev_pad
        
        for h in range(n_H):
            vert_start = h * stride
            vert_end = h * stride + f
            
            for w in range(n_W):
                horiz_start = w * stride
                horiz_end = w * stride + f
                
               
                a_slice_prev = a_prev_pad[vert_start: vert_end, horiz_start: horiz_end, :]

                weights = W[:, :, :]
                z[h, w] = conv_single_step(a_slice_prev, weights)
        
        if flag == 1:
            z_max.append(np.max(z))
        Z.append(z / z_max[i])
        
    cache = (A_prev, W, hparameters)
    
    return Z, z_max, cache

In [5]:
# pooling
def pool_forward(A_prev, hparameters, mode = 'max'):
    m = len(A_prev)
    f = hparameters['f']
    stride = hparameters['stride']
    
    A = list()
    
    for i in range(m):
        (n_H_prev, n_W_prev) = A_prev[i].shape
        
        n_H = int(1 + (n_H_prev - f) / stride)
        n_W = int(1 + (n_W_prev - f) / stride)
        
        a = np.zeros((n_H, n_W))
        
        for h in range(n_H):
            vert_start = h * stride
            vert_end = h * stride + f
            
            for w in range(n_W):
                horiz_start = w * stride
                horiz_end = w * stride + f
                
                a_prev_slice = A_prev[i][vert_start: vert_end, horiz_start: horiz_end]

                if mode == 'max':
                    a[h, w] = np.max(a_prev_slice)
                elif mode == 'avg':
                    a[h, w] = np.mean(a_prev_slice)
                        
        A.append(a)

    cache = (A_prev, hparameters)
    
    return A, cache

In [6]:
# First layer

def borders(images, filter_size = 1, pad = 0, stride = 1, pool_stride = 2, pool_size = 2, z_max = []):
    Wv = edges(filter_size, 'vert')
    hparameters = {'pad': pad, 'stride': stride, 'pool_stride': pool_stride, 'f': pool_size, 'z_max': z_max}
    Z, z_max_v, _ = conv_forward(images, Wv, hparameters)
    
    print('edge filter applied')
    
    hparameters_pool = {'stride': pool_stride, 'f': pool_size}
    Av, _ = pool_forward(Z, hparameters_pool, mode = 'max')
    
    print('vertical filter applied')
    
    Wh = edges(filter_size, 'horiz')
    hparameters = {'pad': pad, 'stride': stride, 'pool_stride': pool_stride, 'f': pool_size, 'z_max': z_max}
    Z, z_max_h, _ = conv_forward(images, Wh, hparameters)
    
    print('edge filter applied')
    
    hparameters_pool = {'stride': pool_stride, 'f': pool_size}
    Ah, _ = pool_forward(Z, hparameters_pool, mode = 'max')
    
    print('horizontal filter applied')   
    
    return [(Av[i] + Ah[i]) / 2 for i in range(len(Av))], list(map(np.max, zip(z_max_v, z_max_h)))

In [69]:
start_time = time.time()

directory = 'C:\\'
img_names = os.listdir(directory)[500:600]
images = list()
for img in img_names:
    images.append(plt.imread(directory + img))
    
len(images)

print("--- %s seconds ---" % (time.time() - start_time))

--- 12.022917985916138 seconds ---


In [70]:
start_time = time.time()

z_max = []

img, z_max = borders(images, filter_size = 2, stride = 3, pool_stride = 2, pool_size = 2)
print('original image borders is calculated')

edge filter applied
vertical filter applied
edge filter applied
horizontal filter applied
original image borders is calculated


In [71]:
blurred_img = [cv2.GaussianBlur(x, (57, 57), 0) for x in images]
print('images blurred')

images blurred


In [72]:
blurred, z_max = borders(blurred_img, filter_size = 2, stride = 2, pool_stride = 2, pool_size = 2, z_max = z_max)
print('blurred image borders is calculated')

print("--- %s seconds ---" % (time.time() - start_time))

edge filter applied
vertical filter applied
edge filter applied
horizontal filter applied
blurred image borders is calculated
--- 575.1750245094299 seconds ---


In [73]:
freq = [np.mean(orig) / np.mean(blurred)for (orig, blurred) in zip(img, blurred)]

In [ ]:
clar = pd.DataFrame([(1 if np.max(f) > 1 else 0, np.mean(f)) for f in freq], columns = ['is_unclarity', 'clarity'])
clar['names'] = img_names
clar = clar.sort_values('clarity').reset_index().reset_index()
clar['file_name'] = '0000'
clar.loc[(clar['level_0'] >= 10) & (clar['level_0'] < 100), 'file_name'] = '000'
clar.loc[(clar['level_0'] >= 100) & (clar['level_0'] < 1000), 'file_name'] = '00'
clar.loc[(clar['level_0'] >= 1000) & (clar['level_0'] < 10000), 'file_name'] = '0'
clar['level_0'] = clar['level_0'].astype(str)
clar['file_name'] = clar['file_name'] + clar['level_0'] + '-' + clar['names']
clar = clar.sort_values('names').set_index('index')
clar

In [58]:
directory_to_save = 'C:\\'

for i in range(len(images)): 
    plt.imsave(directory_to_save + clar.loc[i, 'file_name'], images[i])

print('All images are saved!!!')

All images are saved!!!
